In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import cv2
import imagesize
import PIL
from tqdm import tqdm
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import rcParams

from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from tensorflow import keras

import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import preprocess_input

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from tensorflow.keras.utils import to_categorical


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
root = '/content/drive/MyDrive/ML_Project'
train_csv = root + '/train.csv'
train_images = root + '/train_images'
test_images = root + '/test_images'

In [ ]:
#Read in training data from .csv and address the specie labelling errors
train_df = pd.read_csv(train_csv)
train_df['image_path'] = train_images +'/'+ train_df['image']

train_df.loc[train_df.species.str.contains('beluga'), 'species'] = 'beluga_whale'
train_df.loc[train_df.species.str.contains('globis'), 'species'] = 'globis_whale'

train_df.loc[train_df.species.str.contains('dolpin'), 'species'] = 'bottlenose_dolphin'
train_df.loc[train_df.species.str.contains('kiler'), 'species'] = 'killer_whale'

print(f'List of Unique Species:\n {train_df.species.unique()}')

print(f'\n Number of individual species, updated: {train_df.species.nunique()} \n')

In [ ]:
def imageLoad(paths):
  
  X_train = np.zeros((len(paths), 56, 80, 3))
  for i in tqdm(range(len(paths))):

    #Open image from path
    img = image.load_img(paths[i], target_size = (56, 80), color_mode = 'rgb')
    #Convert to array
    x = image.img_to_array(img)
    #Expand dimensions
    x = np.expand_dims(x, axis = 0)
    #Preprocess input for keras model
    x = preprocess_input(x)
    X_train[i] = x
  return X_train


In [ ]:
#Read in all augmented images. 
augImg_df = pd.read_csv(root + '/final_generated_train_images.csv')
augImg_df = augImg_df[['individual_id','species', 'image_path']]
print(f'Number of individual IDs: {augImg_df.individual_id.nunique()}')


In [ ]:
train_df = train_df.sort_values(by=['individual_id']).reset_index(drop = True)

#Get three images for each individual ID
train_df = train_df.groupby(by=['individual_id']).head(3).reset_index(drop = True)


In [ ]:
#Add the augmented images
train_total_df = train_df.append(augImg_df).reset_index(drop = True)

#Redundancy to make sure there are only 3 images per ID
train_total_df = train_total_df.groupby(by=['individual_id']).head(3).reset_index(drop = True)

In [ ]:
ID= pd.DataFrame(train_total_df.individual_id.value_counts())

ID_df = pd.DataFrame(columns = ['individual_id', 'image_freq'])

ID_df['individual_id'] = ID.index
ID_df['image_freq'] = ID.values.astype(int)

print(ID_df)

In [ ]:
#Sanity check. Should have 15587 classes
print(train_total_df['individual_id'].nunique())

In [ ]:
pd.set_option('display.max_colwidth', None)
#Split the training set into a validation set of 1 image per ID
valid_set = train_total_df.groupby('individual_id').head(1).sort_values(by = ['individual_id']).reset_index(drop = True)
print(valid_set)
train_df2 = train_total_df[~train_total_df.index.isin(valid_set.index)].sort_values(by = ['individual_id']).reset_index(drop = True)
print(train_df2)


In [ ]:
#Load the training set
Xtrain = imageLoad(train_df2['image_path'])

In [ ]:
#Load the validation set
Valid_X = imageLoad(valid_set['image_path'])

In [ ]:
#Rescale the array values to be betweeon 0-1
Xtrain /= 255
Valid_X /=255

In [ ]:
#Function for OHE labels
def labels(y):
    values = np.array(y)
    label_encoder = LabelEncoder()
    label_encoder = label_encoder.fit(values)
    lEncoded = label_encoder.transform(values)
    onehot_encoder = OneHotEncoder(sparse=False)
    lEncoded = lEncoded.reshape(len(lEncoded), 1)
    onehot_encoder = onehot_encoder.fit(lEncoded)
    OHEncoded = onehot_encoder.transform(lEncoded)
    y = OHEncoded
    return y

In [ ]:
#OHE target data for both training and validation

ytrain = train_df2['individual_id']
Ytrain = labels(ytrain)

valid_y = valid_set['individual_id']
Valid_Y = labels(valid_y)

In [ ]:
#from keras import backend as K 
def CNNmodel(learning_rate, epochs, batch_size, dropout):
#MODEL V2
  mod = Sequential()

  # First convolution extracts 16 filters that are 3x3
  # Convolution is followed by max-pooling layer with a 2x2 window
  mod.add(Conv2D(16, 3, activation = 'relu', input_shape = (56, 80, 3)))
  mod.add(BatchNormalization(axis = 3))
  mod.add(MaxPooling2D(pool_size = (2,2)))

  # Second convolution extracts 32 filters that are 3x3
  # Convolution is followed by max-pooling layer with a 2x2 window
  mod.add(Conv2D(32, 3, activation='relu'))
  mod.add(BatchNormalization(axis = 3))
  mod.add(MaxPooling2D(pool_size = (2,2)))

  # Third convolution extracts 64 filters that are 3x3
  # Convolution is followed by max-pooling layer with a 2x2 window
  mod.add(Conv2D(64, 3, activation='relu'))
  mod.add(AveragePooling2D(pool_size = (2,2)))
  mod.add(Dropout(dropout))

  # Flatten feature map to a 1-dim tensor so we can add fully connected layers
  mod.add(Flatten())

  # Create a fully connected layer with ReLU activation and 256 hidden units
  mod.add(Dense(256, activation="relu"))
  mod.add(Dropout(dropout))

  # Create output layer with 15587 nodes and softmax activation
  mod.add(Dense(train_df2['individual_id'].nunique(), activation='softmax'))

  # Create model:
  opt = tf.keras.optimizers.Adam(learning_rate=learning_rate) 

  #model.summary()
  mod.compile(loss = 'categorical_crossentropy', 
                optimizer = opt, 
                metrics = ['acc'])

  callbacks = [ModelCheckpoint(root+"/model_checkpoint", monitor='acc', )]

  history = mod.fit(Xtrain, y = Ytrain,
        batch_size = batch_size,
        epochs = epochs,
        verbose = 1,
        callbacks = callbacks,
        validation_data = (Valid_X, Valid_Y)
        )
  return mod

In [ ]:
#HPO Implementation - BO-TPE
import pickle
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score
from scikeras.wrappers import KerasClassifier

def objective(params):

    params = {
        'learning_rate': abs(int(params['learning_rate'])),
        'batch_size': abs(int(params['batch_size'])),
        'epochs': abs(int(params['epochs'])),
        'dropout': abs(int(params['dropout']))
    }
    clf = KerasClassifier(CNNmodel,**params, verbose = 0)

    score = -np.mean(cross_val_score(clf, Valid_X, Valid_Y, cv=3, 
                                    scoring="accuracy")) #ran on the validation data as it is as small as possible (only one image per class)

    return {'loss': score, 'status': STATUS_OK }

# Define the hyperparameter configuration space
space = {
    'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.1)),
    'batch_size': hp.choice('batch_size', [32, 64, 128, 256]),
    'epochs': hp.quniform('epochs', 20, 60, 10),
    'dropout': hp.quniform('dropout', 0.2, 0.5, 0.1),
}

trials = Trials() #implement trials in order to save the parameter data 

best = fmin(fn=objective, #function to minimize 
            space=space, #hyperparameter space 
            algo=tpe.suggest, #TPE 
            max_evals=5,
            trials=trials
            ) #performs 5 trials inital trials

pickle.dump(trials,  open('trials.p', 'wb'))  # dump data to file
trials = pickle.load(open("trials.p",'rb')) #load it again to trials 

best = fmin(fn=objective, #function to minimize 
            space=space, #hyperparameter space 
            algo=tpe.suggest, #TPE 
            max_evals=10,
            trials=trials
            ) #performs another 5 trials to reach 10 trials total 

print("CNN: Hyperopt estimated optimum {}".format(best))